

```
# This is formatted as code
set up

In [2]:
# Pytorch version change
! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html

# . Install Pytorch image processing library
! pip install kornia==0.5.0

# Copy of CLIP related code
! git clone https://github.com/openai/CLIP.git
%cd /content/CLIP/

# 4.Modeling CLIP
! pip install ftfy regex
import clip
model, preprocess = clip.load('ViT-B/32', jit=False)
model = model.eval()
# 5. Modeling DALL-E
! pip install DALL-E
from dall_e import map_pixels, unmap_pixels, load_model
dec = load_model("https://cdn.openai.com/dall-e/decoder.pkl", 'cuda')


Unable to create process using 'C:\Users\Ram Naryan\anaconda3\New folder\python.exe "C:\Users\Ram Naryan\anaconda3\New folder\Scripts\pip-script.py" install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html'
Unable to create process using 'C:\Users\Ram Naryan\anaconda3\New folder\python.exe "C:\Users\Ram Naryan\anaconda3\New folder\Scripts\pip-script.py" install kornia==0.5.0'
fatal: destination path 'CLIP' already exists and is not an empty directory.

[WinError 3] The system cannot find the path specified: '/content/CLIP/'
c:\Users\Ram Naryan\Documents



Unable to create process using 'C:\Users\Ram Naryan\anaconda3\New folder\python.exe "C:\Users\Ram Naryan\anaconda3\New folder\Scripts\pip-script.py" install ftfy regex'


ModuleNotFoundError: No module named 'clip'

# library imports & function definitions


In [ ]:
import torch
import numpy as np
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as T
import kornia
import PIL
import os, io, sys
import random
import imageio
from IPython import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from google.colab import output
import requests

# Initial setting
im_shape = [512, 512, 3]
sideX, sideY, channels = im_shape
target_image_size = sideX
tau_value = 2.
# Display/save image
def displ(img):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1, 2, 0))
  imageio.imwrite('output.png', np.array(img))
  return display.Image('output.png')

# Random image clipping
def augment(out, cutn=16):
  p_s = []
  for ch in range(cutn):
    sizey = int(torch.zeros(1,).uniform_(.5, .99)*sideY)
    sizex = int(torch.zeros(1,).uniform_(.5, .99)*sideX)
    offsetx = torch.randint(0, sideX - sizex, ())
    offsety = torch.randint(0, sideY - sizey, ())
    apper = out[:, :, offsetx:offsetx + sizex, offsety:offsety + sizey]
    apper = apper + .1*torch.rand(1,1,1,1).cuda()*torch.randn_like(apper, requires_grad=True)
    apper = torch.nn.functional.interpolate(apper, (224,224), mode='bilinear')
    p_s.append(apper)
  into = augs(torch.cat(p_s, 0))
  return into


# normalization and rotation settings
nom = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
augs = kornia.augmentation.RandomRotation(30).cuda()


# extract feature vector from text

In [ ]:
# input text
text_input = 'a beautiful and mysterious house designed by Escher'
# convert text to feature vector
token = clip.tokenize(text_input)
text_v = model.encode_text(token.cuda()).detach().clone()

In [ ]:
# [Check] shape of token, text_v
print('token.shape = ', token.shape)
print('token = ', token)
print('text_v.shape = ', text_v.shape)

token.shape =  torch.Size([1, 77])
token =  tensor([[49406,   320,  1215,   537, 12650,  1212,  4486,   638,    68, 21299,
         49407,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)
text_v.shape =  torch.Size([1, 512])


# Setting parameters and optimization methods

In [ ]:
# set parameters
class Pars(torch.nn.Module):
    def __init__(self):
        super(Pars, self).__init__()
        hots = torch.nn.functional.one_hot((torch.arange(0, 8192).to(torch.int64)), num_classes=8192)
        rng = torch.zeros(1, 64*64, 8192).uniform_()
        for i in range(64*64):
            rng[0,i] = hots[[np.random.randint(8191)]]
        rng = rng.permute(0, 2, 1)
        self.normu = torch.nn.Parameter(rng.cuda().view(1, 8192, 64*64))

    def forward(self):
      normu = torch.nn.functional.gumbel_softmax(self.normu.reshape(1,64*64,8192), dim=1, tau=tau_value).view(1, 8192, 64, 64)
      return normu
# set optimization method
latent = Pars().cuda()
param = [latent.normu]
optimizer = torch.optim.Adam([{'params': param, 'lr': .01}])


In [ ]:
# [Check] Image generation from parameters
with torch.no_grad():
  out = unmap_pixels(torch.sigmoid(dec(latent())[:, :3].float()))
  displ(out.cpu()[0])

  print('latent().shape = ', latent().shape)
  print('dec(latent()).shape = ', dec(latent()).shape)
  print('out.shape = ', out.shape)

# 学習

In [ ]:
# 学習ループ
for iteration in range(1001):

  # --- 順伝播 ---
  # パラメータから画像を生成
  out = unmap_pixels(torch.sigmoid(dec(latent())[:, :3].float()))
  # 画像をランダム切り出し・回転
  into = augment(out)
  # 画像を正規化
  into = nom((into))
  # 画像から特徴ベクトルを取得
  image_v = model.encode_image(into)
  # テキストと画像の特徴ベクトルのCOS類似度を計算
  loss = -torch.cosine_similarity(text_v, image_v).mean()

  # 逆伝播
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  # 学習率の調整
  for g in optimizer.param_groups:
    g['lr'] = g['lr']*1.005
    g['lr'] = min(g['lr'], .12)

  # ログ表示
  if iteration % 50 == 0:
    with torch.no_grad():

      # 生成画像の表示・保存
      out = unmap_pixels(torch.sigmoid(dec(latent())[:, :3]).float())  ###
      displ(out.cpu()[0])  ###

      # データ表示
      print('iter = ',iteration)
      for g in optimizer.param_groups:
        print('lr = ', g['lr'])
      print('tau_value = ', tau_value)
      print('loss = ',loss.item())
      print('\n')
